In [1]:
# para gestionar ficheros
import os
import pathlib

# para dataframes
import pandas as pd
import numpy as np

# para pintar
import pylab as plt
import seaborn as sns
## para que salga el grafico
%matplotlib inline

# para la conexión SQL
import mysql.connector as conn
from sqlalchemy import create_engine


# display config
pd.options.display.max_rows = None 
pd.options.display.max_columns = None

In [2]:
# ----------- DOWNCAST ------------- #
def downcast_types(df):
    for columna in df.select_dtypes('object').columns: 
        df[columna]=df[columna].astype('category')
    for columna in df.select_dtypes(np.number).columns: 
        df[columna]=pd.to_numeric(df[columna], downcast='integer')

In [3]:
def eliminar_horas(column):
    column = column.replace(r"\ [0-9]{2}:[0-9]{2}:[0-9]{2}", value = "", regex = True)
    return column

In [4]:
def normalizar_fechas(column):
    column = eliminar_horas(column)
    column = pd.to_datetime(column)
    return column

In [5]:
def eliminar_duplicados(df):
    df = df.drop_duplicates().reset_index(drop = True)

In [6]:
dir_path = pathlib.Path('/Users/carlosbasco/Desktop/Projects/SQL-Database-Project/data')
listaFicheros = os.listdir(dir_path) 
#Obtenemos todos los ficheros a leer

In [7]:
dict_files = {}
for fichero in dir_path.iterdir():
    dict_files[fichero.name] = pd.read_csv(fichero.open(), encoding= "UTF-8")
print(dict_files.keys(), len(dict_files.keys()))

dict_keys(['actor.csv', 'film.csv', 'language.csv', 'old_HDD.csv', 'category.csv', 'rental.csv', 'inventory.csv']) 7


In [8]:
#### LIMPIEZA PRIMER CSV ####
df = dict_files['actor.csv'] 

In [9]:
df.head()

,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2006-02-15 04:34:33
1,2,NICK,WAHLBERG,2006-02-15 04:34:33
2,3,ED,CHASE,2006-02-15 04:34:33
3,4,JENNIFER,DAVIS,2006-02-15 04:34:33
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15 04:34:33


In [10]:
df.shape

(200, 4)

In [11]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   actor_id     200 non-null    int64 
 1   first_name   200 non-null    object
 2   last_name    200 non-null    object
 3   last_update  200 non-null    object
dtypes: int64(1), object(3)
memory usage: 41.0 KB


In [12]:
df["full_name"] = df.first_name + ' ' + df.last_name

In [13]:
df.full_name.value_counts()

SUSAN DAVIS             2
EWAN GOODING            1
DARYL CRAWFORD          1
GRETA KEITEL            1
JANE JACKMAN            1
ADAM HOPPER             1
RICHARD PENN            1
GENE HOPKINS            1
RITA REYNOLDS           1
ED MANSFIELD            1
MORGAN WILLIAMS         1
LUCILLE DEE             1
WHOOPI HURT             1
GEOFFREY HESTON         1
CATE HARRIS             1
JADA RYDER              1
RIVER DEAN              1
ANGELA WITHERSPOON      1
KIM ALLEN               1
ALBERT JOHANSSON        1
FAY WINSLET             1
EMILY DEE               1
RUSSELL TEMPLE          1
JAYNE NOLTE             1
CATE MCQUEEN            1
PENELOPE GUINESS        1
FRANCES TOMEI           1
ALBERT NOLTE            1
MATTHEW LEIGH           1
PENELOPE CRONYN         1
SIDNEY CROWE            1
GROUCHO DUNST           1
GINA DEGENERES          1
WARREN NOLTE            1
SYLVESTER DERN          1
CAMERON ZELLWEGER       1
RUSSELL BACALL          1
MORGAN HOPKINS          1
MORGAN MCDOR

In [14]:
df[df.full_name == 'SUSAN DAVIS']

,actor_id,first_name,last_name,last_update,full_name
100,101,SUSAN,DAVIS,2006-02-15 04:34:33,SUSAN DAVIS
109,110,SUSAN,DAVIS,2006-02-15 04:34:33,SUSAN DAVIS


In [15]:
df = df.drop(df.index[109]).reset_index(drop = True)

In [16]:
df[df.full_name == 'SUSAN DAVIS']

,actor_id,first_name,last_name,last_update,full_name
100,101,SUSAN,DAVIS,2006-02-15 04:34:33,SUSAN DAVIS


In [17]:
eliminar_duplicados(df)

In [18]:
df.last_update = normalizar_fechas(df.last_update)

In [19]:
df.head()

,actor_id,first_name,last_name,last_update,full_name
0,1,PENELOPE,GUINESS,2006-02-15,PENELOPE GUINESS
1,2,NICK,WAHLBERG,2006-02-15,NICK WAHLBERG
2,3,ED,CHASE,2006-02-15,ED CHASE
3,4,JENNIFER,DAVIS,2006-02-15,JENNIFER DAVIS
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15,JOHNNY LOLLOBRIGIDA


In [20]:
dictio = zip(df.full_name,df.actor_id)
leyenda_actores=dict(dictio) #Esta leyenda la usaremos más adelante

In [21]:
leyenda_actores

{'PENELOPE GUINESS': 1,
 'NICK WAHLBERG': 2,
 'ED CHASE': 3,
 'JENNIFER DAVIS': 4,
 'JOHNNY LOLLOBRIGIDA': 5,
 'BETTE NICHOLSON': 6,
 'GRACE MOSTEL': 7,
 'MATTHEW JOHANSSON': 8,
 'JOE SWANK': 9,
 'CHRISTIAN GABLE': 10,
 'ZERO CAGE': 11,
 'KARL BERRY': 12,
 'UMA WOOD': 13,
 'VIVIEN BERGEN': 14,
 'CUBA OLIVIER': 15,
 'FRED COSTNER': 16,
 'HELEN VOIGHT': 17,
 'DAN TORN': 18,
 'BOB FAWCETT': 19,
 'LUCILLE TRACY': 20,
 'KIRSTEN PALTROW': 21,
 'ELVIS MARX': 22,
 'SANDRA KILMER': 23,
 'CAMERON STREEP': 24,
 'KEVIN BLOOM': 25,
 'RIP CRAWFORD': 26,
 'JULIA MCQUEEN': 27,
 'WOODY HOFFMAN': 28,
 'ALEC WAYNE': 29,
 'SANDRA PECK': 30,
 'SISSY SOBIESKI': 31,
 'TIM HACKMAN': 32,
 'MILLA PECK': 33,
 'AUDREY OLIVIER': 34,
 'JUDY DEAN': 35,
 'BURT DUKAKIS': 36,
 'VAL BOLGER': 37,
 'TOM MCKELLEN': 38,
 'GOLDIE BRODY': 39,
 'JOHNNY CAGE': 40,
 'JODIE DEGENERES': 41,
 'TOM MIRANDA': 42,
 'KIRK JOVOVICH': 43,
 'NICK STALLONE': 44,
 'REESE KILMER': 45,
 'PARKER GOLDBERG': 46,
 'JULIA BARRYMORE': 47,
 'FRANCES

In [22]:
downcast_types(df)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   actor_id     199 non-null    int16         
 1   first_name   199 non-null    category      
 2   last_name    199 non-null    category      
 3   last_update  199 non-null    datetime64[ns]
 4   full_name    199 non-null    category      
dtypes: category(3), datetime64[ns](1), int16(1)
memory usage: 22.8 KB


In [24]:
dict_files['actor.csv'] = df

In [25]:
#### LIMPIEZA SEGUNDO CSV ####
df = dict_files['film.csv']

In [26]:
df.head()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,NaN,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,NaN,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,NaN,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 05:03:42
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,NaN,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2006-02-15 05:03:42
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,NaN,6,2.99,130,22.99,G,Deleted Scenes,2006-02-15 05:03:42


In [27]:
eliminar_duplicados(df)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   film_id               1000 non-null   int64  
 1   title                 1000 non-null   object 
 2   description           1000 non-null   object 
 3   release_year          1000 non-null   int64  
 4   language_id           1000 non-null   int64  
 5   original_language_id  0 non-null      float64
 6   rental_duration       1000 non-null   int64  
 7   rental_rate           1000 non-null   float64
 8   length                1000 non-null   int64  
 9   replacement_cost      1000 non-null   float64
 10  rating                1000 non-null   object 
 11  special_features      1000 non-null   object 
 12  last_update           1000 non-null   object 
dtypes: float64(3), int64(5), object(5)
memory usage: 101.7+ KB


In [29]:
df = df.drop(['original_language_id'], axis=1)
df.head()

,film_id,title,description,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 05:03:42
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2006-02-15 05:03:42
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,6,2.99,130,22.99,G,Deleted Scenes,2006-02-15 05:03:42


In [30]:
df.rating.unique()

array(['PG', 'G', 'NC-17', 'PG-13', 'R'], dtype=object)

In [31]:
dictio = zip(df.title, df.film_id)
leyenda_films=dict(dictio) #Esta leyenda la usaremos más adelante

In [32]:
df.last_update = normalizar_fechas(df.last_update)

In [33]:
downcast_types(df)

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   film_id           1000 non-null   int16         
 1   title             1000 non-null   category      
 2   description       1000 non-null   category      
 3   release_year      1000 non-null   int16         
 4   language_id       1000 non-null   int8          
 5   rental_duration   1000 non-null   int8          
 6   rental_rate       1000 non-null   float64       
 7   length            1000 non-null   int16         
 8   replacement_cost  1000 non-null   float64       
 9   rating            1000 non-null   category      
 10  special_features  1000 non-null   category      
 11  last_update       1000 non-null   datetime64[ns]
dtypes: category(4), datetime64[ns](1), float64(2), int16(3), int8(2)
memory usage: 118.3 KB


In [35]:
dict_files['film.csv'] = df

In [36]:
#### LIMPIEZA TERCER CSV ####
df = dict_files['language.csv']

In [37]:
df

,language_id,name,last_update
0,1,English,2006-02-15 05:02:19
1,2,Italian,2006-02-15 05:02:19
2,3,Japanese,2006-02-15 05:02:19
3,4,Mandarin,2006-02-15 05:02:19
4,5,French,2006-02-15 05:02:19
5,6,German,2006-02-15 05:02:19


In [38]:
df.last_update = normalizar_fechas(df.last_update)

In [39]:
df.head(10)

,language_id,name,last_update
0,1,English,2006-02-15
1,2,Italian,2006-02-15
2,3,Japanese,2006-02-15
3,4,Mandarin,2006-02-15
4,5,French,2006-02-15
5,6,German,2006-02-15


In [40]:
downcast_types(df)

In [41]:
dict_files['language.csv'] = df

In [42]:
#### LIMPIEZA CUARTO CSV ####
df = dict_files['old_HDD.csv']

In [43]:
df.head(15) #Vamos a usar el autor_id y el title_id en lugar de tener columnas duplicadas con la informacion

,first_name,last_name,title,release_year,category_id
0,PENELOPE,GUINESS,ACADEMY DINOSAUR,2006,6
1,PENELOPE,GUINESS,ANACONDA CONFESSIONS,2006,2
2,PENELOPE,GUINESS,ANGELS LIFE,2006,13
3,PENELOPE,GUINESS,BULWORTH COMMANDMENTS,2006,10
4,PENELOPE,GUINESS,CHEAPER CLYDE,2006,14
5,PENELOPE,GUINESS,COLOR PHILADELPHIA,2006,4
6,PENELOPE,GUINESS,ELEPHANT TROJAN,2006,11
7,PENELOPE,GUINESS,GLEAMING JAWBREAKER,2006,15
8,PENELOPE,GUINESS,HUMAN GRAFFITI,2006,10
9,PENELOPE,GUINESS,KING EVOLUTION,2006,8


In [44]:
df["full_name"]=df.first_name + ' ' + df.last_name

In [45]:
df.replace({"title":leyenda_films}, inplace=True)
df.replace({"full_name":leyenda_actores}, inplace=True)
df.rename(columns = {'title':'film_id', 'full_name':'actor_id'}, inplace = True)

In [46]:
df = df.drop(['first_name','last_name','release_year'], axis=1)

In [47]:
eliminar_duplicados(df)

In [48]:
downcast_types(df)

In [49]:
dict_files['old_HDD.csv'] = df

In [50]:
#### LIMPIEZA QUINTO CSV ####
df = dict_files['category.csv']

In [51]:
df.head(20)

,category_id,name,last_update
0,1,Action,2006-02-15 04:46:27
1,2,Animation,2006-02-15 04:46:27
2,3,Children,2006-02-15 04:46:27
3,4,Classics,2006-02-15 04:46:27
4,5,Comedy,2006-02-15 04:46:27
5,6,Documentary,2006-02-15 04:46:27
6,7,Drama,2006-02-15 04:46:27
7,8,Family,2006-02-15 04:46:27
8,9,Foreign,2006-02-15 04:46:27
9,10,Games,2006-02-15 04:46:27


In [52]:
df.last_update = eliminar_horas(df.last_update)
df.last_update = pd.to_datetime(df.last_update)

In [53]:
df.head(20)

,category_id,name,last_update
0,1,Action,2006-02-15
1,2,Animation,2006-02-15
2,3,Children,2006-02-15
3,4,Classics,2006-02-15
4,5,Comedy,2006-02-15
5,6,Documentary,2006-02-15
6,7,Drama,2006-02-15
7,8,Family,2006-02-15
8,9,Foreign,2006-02-15
9,10,Games,2006-02-15


In [54]:
dict_files['category.csv'] = df

In [55]:
#### LIMPIEZA SEXTO CSV ####
df = dict_files['rental.csv'] # Sacar cuanto tiempo tardan en devolver las películas y si vuelven o no los customers

In [56]:
df.head(100)

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53
5,6,2005-05-24 23:08:07,2792,549,2005-05-27 01:32:07,1,2006-02-15 21:30:53
6,7,2005-05-24 23:11:53,3995,269,2005-05-29 20:34:53,2,2006-02-15 21:30:53
7,8,2005-05-24 23:31:46,2346,239,2005-05-27 23:33:46,2,2006-02-15 21:30:53
8,9,2005-05-25 00:00:40,2580,126,2005-05-28 00:22:40,1,2006-02-15 21:30:53
9,10,2005-05-25 00:02:21,1824,399,2005-05-31 22:44:21,2,2006-02-15 21:30:53


In [57]:
eliminar_duplicados(df)

In [58]:
df = df.drop(['last_update'], axis=1)

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   rental_id     1000 non-null   int64 
 1   rental_date   1000 non-null   object
 2   inventory_id  1000 non-null   int64 
 3   customer_id   1000 non-null   int64 
 4   return_date   1000 non-null   object
 5   staff_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


In [60]:
df.rental_date = normalizar_fechas(df.rental_date)
df.return_date = normalizar_fechas(df.return_date)

In [61]:
downcast_types(df)

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   rental_id     1000 non-null   int16         
 1   rental_date   1000 non-null   datetime64[ns]
 2   inventory_id  1000 non-null   int16         
 3   customer_id   1000 non-null   int16         
 4   return_date   1000 non-null   datetime64[ns]
 5   staff_id      1000 non-null   int8          
dtypes: datetime64[ns](2), int16(3), int8(1)
memory usage: 22.6 KB


In [63]:
dict_files['rental.csv'] = df

In [64]:
#### LIMPIEZA SEPTIMO CSV ####
df = dict_files['inventory.csv']

In [65]:
df.head(100)

,inventory_id,film_id,store_id,last_update
0,1,1,1,2006-02-15 05:09:17
1,2,1,1,2006-02-15 05:09:17
2,3,1,1,2006-02-15 05:09:17
3,4,1,1,2006-02-15 05:09:17
4,5,1,2,2006-02-15 05:09:17
5,6,1,2,2006-02-15 05:09:17
6,7,1,2,2006-02-15 05:09:17
7,8,1,2,2006-02-15 05:09:17
8,9,2,2,2006-02-15 05:09:17
9,10,2,2,2006-02-15 05:09:17


In [66]:
df.shape

(1000, 4)

In [67]:
eliminar_duplicados(df)

In [68]:
df.last_update = normalizar_fechas(df.last_update)

In [69]:
downcast_types(df)

In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   inventory_id  1000 non-null   int16         
 1   film_id       1000 non-null   int16         
 2   store_id      1000 non-null   int8          
 3   last_update   1000 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int16(2), int8(1)
memory usage: 12.8 KB


In [71]:
df = df.drop(['inventory_id',], axis=1)

In [72]:
df

,film_id,store_id,last_update
0,1,1,2006-02-15
1,1,1,2006-02-15
2,1,1,2006-02-15
3,1,1,2006-02-15
4,1,2,2006-02-15
5,1,2,2006-02-15
6,1,2,2006-02-15
7,1,2,2006-02-15
8,2,2,2006-02-15
9,2,2,2006-02-15


In [73]:
#df = df.groupby(df.store_id).value_counts() #RECORDATORIO Agruparemos las películas por almacen
df.reset_index()

,index,film_id,store_id,last_update
0,0,1,1,2006-02-15
1,1,1,1,2006-02-15
2,2,1,1,2006-02-15
3,3,1,1,2006-02-15
4,4,1,2,2006-02-15
5,5,1,2,2006-02-15
6,6,1,2,2006-02-15
7,7,1,2,2006-02-15
8,8,2,2,2006-02-15
9,9,2,2,2006-02-15


In [74]:
dict_files['inventory.csv'] = df

In [84]:
#### CONEXION ####

with open("pass/password.txt") as archivo:
    host_line = archivo.readline().rstrip('\n')
    user_line = archivo.readline().rstrip('\n')
    passwd_line = archivo.readline().rstrip('\n')

In [85]:
#### CREAMOS UNA NUEVA BASE DE DATOS ####

str_conn='mysql+pymysql://'+user_line+':'+passwd_line+'@'+host_line+':3306'

cursor=create_engine(str_conn)

cursor.execute('drop database if exists video_store;')
cursor.execute('create database video_store;')

In [95]:
#### GENERAMOS LAS TABLAS DE SQL ####

str_conn='mysql+pymysql://'+user_line+':'+passwd_line+'@'+host_line+':3306'+'/video_store'

cursor=create_engine(str_conn)

for element in dict_files:
    name_aux = (element.split('.'))[0]
    dict_files[element].to_sql(name=name_aux, con=cursor, if_exists='replace', index=False)